In [56]:
from __future__ import print_function
import keras

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import math


In [36]:
train_data_dir = './input/train'
test_data_dir = './input/test'
val_data_dir = './input/val'

In [37]:
nb_train_samples = 5217
nb_validation_samples = 17
epochs = 15
batch_size = 8

In [38]:
# dimensions of our images.
img_width, img_height = 150, 150

input_shape = (img_width, img_height, 3)
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)

In [39]:
train_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [40]:
test_gen = ImageDataGenerator(rescale=1. / 255)

In [41]:
train_generator = train_gen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [42]:
test_generator = test_gen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 624 images belonging to 2 classes.


In [43]:
validation_generator = test_gen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 16 images belonging to 2 classes.


In [44]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(48, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [45]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [46]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/15
652/652 [==============================] - 207s 318ms/step - loss: 0.4744 - acc: 0.7816 - val_loss: 0.7987 - val_acc: 0.7500
Epoch 2/15
652/652 [==============================] - 200s 307ms/step - loss: 0.3105 - acc: 0.8785 - val_loss: 0.7010 - val_acc: 0.8125
Epoch 3/15
652/652 [==============================] - 203s 311ms/step - loss: 0.2887 - acc: 0.8955 - val_loss: 0.5506 - val_acc: 0.8750
Epoch 4/15
652/652 [==============================] - 197s 302ms/step - loss: 0.2724 - acc: 0.9076 - val_loss: 0.5803 - val_acc: 0.8125
Epoch 5/15
652/652 [==============================] - 198s 304ms/step - loss: 0.2658 - acc: 0.9099 - val_loss: 1.2551 - val_acc: 0.6875
Epoch 6/15
652/652 [==============================] - 207s 318ms/step - loss: 0.2566 - acc: 0.9145 - val_loss: 0.8736 - val_acc: 0.6875
Epoch 7/15
652/652 [==============================] - 206s 315ms/step - loss: 0.2385 - acc: 0.9225 - val_loss: 0.8633 - val_acc: 0.7500
Epoch 8/15
652/652 [============================

In [ ]:
score = model.evaluate_generator(test_generator, math.ceil(5217 / batch_size), verbose=1)

 25/653 [>.............................] - ETA: 1:11

In [ ]:
score